In [8]:
# pip install tabulate

p324

In [9]:
# 문장 분류 모델 

from torch import nn 

class SentenceClassifier(nn.Module):
    def __init__(
        self, 
        n_vocab, 
        hidden_dim,
        embedding_dim, 
        n_layers,
        dropout = 0.5,
        bidirectional = True,
        model_type = "lstm"
    ):

        super().__init__()

        self.embedding = nn.Embedding( # 인스턴스 생성 
            num_embeddings=n_vocab,
            embedding_dim=embedding_dim,
            padding_idx=0
        )

        if model_type == "rnn":
            self.model = nn.RNN(
                input_size = embedding_dim,
                hidden_size=hidden_dim,
                num_layers=n_layers,
                bidirectional=bidirectional,
                dropout=dropout,
                batch_first=True,
            )

        elif model_type == "lstm":
            self.model = nn.LSTM(
                input_size = embedding_dim,
                hidden_size = hidden_dim,
                num_layers=n_layers,
                bidirectional=bidirectional,
                dropout=dropout,
                batch_first=True,
            )

        if bidirectional:
            self.classifier = nn.Linear(hidden_dim * 2,1)
        else:
            self.classifier = nn.Linear(hidden_dim, 1)
        self.dropout = nn.Dropout(dropout)


    def forward(self, inputs): # 순방향 메서드
        embeddings = self.embedding(inputs) # 입력받은 정수 인코딩을 임베딩 계층에 통과시켜 임베딩 값을 얻음
        output, _ = self.model(embeddings) # 얻음 임베딩 값을 모델에 입력하여 출력값 얻음 
        last_output = output[:,-1,:] 
        last_output = self.dropout(last_output)
        logits = self.classifier(last_output) # 마지막 값만 추출하여 분류기 계층에 전달 
        return logits



In [10]:
# 데이터 세트 불러오기 

import pandas as pd
from Korpora import Korpora

corpus = Korpora.load("nsmc")
corpus_df = pd.DataFrame(corpus.test)

train = corpus_df.sample(frac=0.9, random_state=42) # 90%만 뽑아서 train 
test = corpus_df.drop(train.index) # drop한 나머지 10%를 test

print(train.head(5).to_markdown())
print("Training Data Size :", len(train))
print("Testing Data Size :", len(test))


    Korpora 는 다른 분들이 연구 목적으로 공유해주신 말뭉치들을
    손쉽게 다운로드, 사용할 수 있는 기능만을 제공합니다.

    말뭉치들을 공유해 주신 분들에게 감사드리며, 각 말뭉치 별 설명과 라이센스를 공유 드립니다.
    해당 말뭉치에 대해 자세히 알고 싶으신 분은 아래의 description 을 참고,
    해당 말뭉치를 연구/상용의 목적으로 이용하실 때에는 아래의 라이센스를 참고해 주시기 바랍니다.

    # Description
    Author : e9t@github
    Repository : https://github.com/e9t/nsmc
    References : www.lucypark.kr/docs/2015-pyconkr/#39

    Naver sentiment movie corpus v1.0
    This is a movie review dataset in the Korean language.
    Reviews were scraped from Naver Movies.

    The dataset construction is based on the method noted in
    [Large movie review dataset][^1] from Maas et al., 2011.

    [^1]: http://ai.stanford.edu/~amaas/data/sentiment/

    # License
    CC0 1.0 Universal (CC0 1.0) Public Domain Dedication
    Details in https://creativecommons.org/publicdomain/zero/1.0/

[Korpora] Corpus `nsmc` is already installed at C:\Users\KDP-35\Korpora\nsmc\ratings_train.txt
[Korpora] Corpus `nsmc` is already installed at C:\Users\KD

In [11]:
# 데이터 토큰화 및 단어 사전 구축 

from konlpy.tag import Okt
from collections import Counter

def build_vocab(corpus, n_vocab, special_tokens):
    counter = Counter()
    for tokens in corpus:
        counter.update(tokens)
    vocab = special_tokens
    for token, count in counter.most_common(n_vocab):
        vocab.append(token)
    return vocab


tokenizer = Okt()
train_tokens = [tokenizer.morphs(review) for review in train.text]
test_tokens = [tokenizer.morphs(review) for review in test.text]

vocab = build_vocab(corpus = train_tokens, n_vocab=5000, special_tokens=["<pad>", "<unk>"]) 
# 문장의 길이를 맞추기 위해 <pad> 토큰 추가, 단어 사전 최대 길이 5000
token_to_id = {token: idx for idx, token in enumerate(vocab)}
id_to_token = {idx: token for idx, token in enumerate(vocab)}

print(vocab[:10])
print(len(vocab))

['<pad>', '<unk>', '.', '이', '영화', '의', '..', '가', '에', '...']
5002


In [12]:
# 정수 인코딩 및 패딩 - 토큰을 정수로 변환 

import numpy as np 

## 너무 긴 문장은 최대 길이로 줄이고 너무 작은 길이는 최대 길이와 동일한 크기로 변환 
def pad_sequences(sequences, max_length, pad_value):
    result = list() # []
    for sequence in sequences:
        sequence = sequence[:max_length]
        pad_length = max_length - len(sequence)
        padded_sequence =  sequence + [pad_value] * pad_length
        result.append(padded_sequence)

    return np.asarray(result)


unk_id = token_to_id["<unk>"]
train_ids = [[token_to_id.get(token, unk_id) for token in review] for review in train_tokens]
test_ids = [[token_to_id.get(token, unk_id) for token in review] for review in test_tokens]

## 패딩 
max_length = 32
pad_id = token_to_id["<pad>"]
train_ids = pad_sequences(train_ids, max_length, pad_id)
test_ids = pad_sequences(test_ids, max_length, pad_id)

print(train_ids[0])
print(test_ids[0])

print(train_ids[2])
print(test_ids[2])


[ 223 1716   10 4036 2095  193  755    4    2 2330 1031  220   26   13
 4839    1    1    1    2    0    0    0    0    0    0    0    0    0
    0    0    0    0]
[3307    5 1997  456    8    1 1013 3906    5    1    1   13  223   51
    3    1 4684    6    0    0    0    0    0    0    0    0    0    0
    0    0    0    0]
[  1  33  76 308   2   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
[   1 1404    1    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0]


-> OOV의 경우 1(<"unk">)로 인코딩된다

In [13]:
# 데이터로더 적용 

import torch 
from torch.utils.data import TensorDataset, DataLoader

train_ids = torch.tensor(train_ids)
test_ids = torch.tensor(test_ids)

train_labels = torch.tensor(train.label.values, dtype = torch.float32)
test_labels = torch.tensor(test.label.values, dtype=torch.float32)

## TensorDataset : pytorch tensor 형태를 입력값으로 받음 
## -> 정수 인코딩과 라벨값을 pytorch tensor 형태로 변환
train_dataset = TensorDataset(train_ids, train_labels)
test_dataset = TensorDataset(test_ids, test_labels)

## 모델 학습 과정 시 각 step 마다 데이터를 batch size 크기로 분할하여 넣어 효과적이고 효율적인 학습 진행하도록 
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

In [14]:
# 손실함수와 최적화 함수 정의 

from torch import optim

n_vocab = len(token_to_id)
hidden_dim = 64             # 은닉 상태 크기 64
embedding_dim = 128         # embedding 벡터 크기 128
n_layers = 2                # 신경망을 2개의 층으로 구성 

device = 'cuda' if torch.cuda.is_available() else "cpu"
classifier = SentenceClassifier(
    n_vocab=n_vocab, hidden_dim=hidden_dim, embedding_dim=embedding_dim, n_layers=n_layers
).to(device)

## 문장의 긍정, 부정을 분류하므로 손실함수는 이진 교차 엔트로피 함수 적용 
## BCEWithLogitsLoss : BCELoss + Sigmoid
criterion = nn.BCEWithLogitsLoss().to(device)   

## RMSprop : 모든 기울기를 누적하지 않고, 지수 가중 이동 평균(EWMA)을 사용해 학습률 조절 
## - 기울기 제곱 값의 평균값이 작아지면 학습률 증가, 반대일 경우 학습률을 감소시켜 불필요한 지역 최솟값에 빠지는 것 방지 
## - 기울기의 크기가 큰 경우에는 빠른 수렴을 보이며 작은 경우에는 더 작은 학습률을 유지시켜 더 안정적으로 최적화 수행 
optimizer = optim.RMSprop(classifier.parameters(), lr=0.001)

In [15]:
# 모델 학습 및 테스트 
# - 모델 학습 중간에 학습이 잘 이뤄지고 있는지 확인하기 위해 일정 배치 학습 후 테스트 데이터세트로 손실값 확인 

def train(model, datasets, criterion, optimizer, device, interval):
    model.train() # train 모드 설정 
    losses = list()

    for step, (input_ids, labels) in enumerate(datasets):
        input_ids = input_ids.to(device)
        labels = labels.to(device).unsqueeze(1) # unsqueeze : 지정한 자리에 size가 1인 빈 공간을 채워주면서 차원을 확장

        logits = model(input_ids)
        loss = criterion(logits, labels)
        losses.append(loss.item())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if step % interval == 0:
            print(f'Train Loss {step} : {np.mean(losses)}')


def test(model, datasets, criterion, device):
    model.eval() # 검증 모드 설정 
    losses = list()
    corrects = list()

    for step, (input_ids, labels) in enumerate(datasets):
        input_ids = input_ids.to(device)
        labels= labels.to(device).unsqueeze(1)
        
        logits = model(input_ids)
        loss= criterion(logits, labels)
        losses.append(loss.item())
        yhat = torch.sigmoid(logits) > .5 
        corrects.extend(
            torch.eq(yhat, labels).cpu().tolist()
        )

    print(f'Val Loss : {np.mean(losses)}, Val Accuracy : {np.mean(corrects)}')


epochs = 5
interval = 500

## 위의 코드에서 정의한 criterion, optimizer 사용 
for epoch in range(epochs):
    train(classifier, train_loader, criterion, optimizer, device, interval)
    test(classifier, test_loader, criterion, device) # 검증은 test 데이터셋으로 진행 

Train Loss 0 : 0.6771101355552673
Train Loss 500 : 0.6929116890340032
Train Loss 1000 : 0.675214636218655
Train Loss 1500 : 0.6573497134991124
Train Loss 2000 : 0.6340165737344288
Train Loss 2500 : 0.611079863039125
Val Loss : 0.48223146953331397, Val Accuracy : 0.7716
Train Loss 0 : 0.319711297750473
Train Loss 500 : 0.46402559532019905
Train Loss 1000 : 0.4583107573645456
Train Loss 1500 : 0.45414992874062593
Train Loss 2000 : 0.44682633556213336
Train Loss 2500 : 0.44069109470700796
Val Loss : 0.417280415709788, Val Accuracy : 0.8038
Train Loss 0 : 0.3209379315376282
Train Loss 500 : 0.37337728475263254
Train Loss 1000 : 0.3746809307095054
Train Loss 1500 : 0.3772474047166359
Train Loss 2000 : 0.37759283012610445
Train Loss 2500 : 0.3751473431823636
Val Loss : 0.3963756580560352, Val Accuracy : 0.8144
Train Loss 0 : 0.38737574219703674
Train Loss 500 : 0.330378498071563
Train Loss 1000 : 0.3296247088014484
Train Loss 1500 : 0.33187238574474753
Train Loss 2000 : 0.33404232089271374
T

-> 테스트 데이터세트에 대해서 손실이 감소하며 정확도 상승 

- 모델 학습 과정에서 임베딩 계층을 비롯한 순환 신경망 내의 여러 가중치 최적화
- 학습된 임베딩 계층의 가중치를 동일한 단어 사전을 사용하는 토큰의 임베딩 값으로 사용 ㅇ 

In [16]:
# 학습된 모델로부터 임베딩 추출 

token_to_embedding = dict()
embedding_matrix = classifier.embedding.weight.detach().cpu().numpy()
# weight (Tensor) 
# – the learnable weights of the module of shape (num_embeddings, embedding_dim) initialized from N(0,1)
# .detach().cpu().numpy() : tensor를 numpy로 변환하기 위해서는 cpu 메모리로 옮겨야 한다, .numpy() 이전에 .cpu()가 실행되야 한다.

for word, emb in zip(vocab, embedding_matrix):
    token_to_embedding[word] = emb

token = vocab[1000]
print(token, token_to_embedding[token])

보고싶다 [-1.1848216   1.3872174   0.5732668   1.1607504  -0.662444    0.99736863
  0.7150054  -0.96393883 -0.9589178   1.6164086  -0.59270656  0.0131274
  0.6605736   0.9091631   0.30927375  0.714278    0.36193472 -1.8440436
  0.01952901  0.43299648 -1.9145285  -1.130032    0.29609177  1.8629019
 -0.12541625 -1.3493133   0.68224716 -0.36267665 -0.4312131  -1.742426
 -0.7756632   0.05902952 -1.1977922   0.92043453 -0.502909   -0.34839186
  0.08734956  1.2330111   0.181254   -0.69788235 -0.7337799   0.5403862
 -0.15272637 -0.7318662   0.2503486  -0.23352377  0.20349824  0.67190075
  0.671132   -1.7439362   0.23648489  0.6276126  -2.5532656  -0.32033324
 -3.0388176  -0.4134272  -1.1825596   1.2390556   0.13895416  0.25890353
 -1.0866429   1.8996519   0.8110003   0.57520425  0.3537721   1.4119903
  1.092689   -0.43305323 -0.67739654  0.16830924 -1.0965645   1.1179385
 -0.01284698 -1.0276898   0.43917525 -0.19183785 -0.5664117  -0.38659865
  1.3136035  -0.37380627 -0.78816944  0.08326602 -0.19

In [17]:
# 사전 학습된 임베딩 값을 초기값으로 적용해 모델 학습시키는 방법 

## 1) p288 영화 리뷰 데이터세트 전처리

import pandas as pd
from Korpora import Korpora
from konlpy.tag import Okt

corpus = Korpora.load("nsmc")
corpus = pd.DataFrame(corpus.test)

tokenizer = Okt() 
tokens = [tokenizer.morphs(review) for review in corpus.text] # 형태소 추출 
print(tokens[:3])


    Korpora 는 다른 분들이 연구 목적으로 공유해주신 말뭉치들을
    손쉽게 다운로드, 사용할 수 있는 기능만을 제공합니다.

    말뭉치들을 공유해 주신 분들에게 감사드리며, 각 말뭉치 별 설명과 라이센스를 공유 드립니다.
    해당 말뭉치에 대해 자세히 알고 싶으신 분은 아래의 description 을 참고,
    해당 말뭉치를 연구/상용의 목적으로 이용하실 때에는 아래의 라이센스를 참고해 주시기 바랍니다.

    # Description
    Author : e9t@github
    Repository : https://github.com/e9t/nsmc
    References : www.lucypark.kr/docs/2015-pyconkr/#39

    Naver sentiment movie corpus v1.0
    This is a movie review dataset in the Korean language.
    Reviews were scraped from Naver Movies.

    The dataset construction is based on the method noted in
    [Large movie review dataset][^1] from Maas et al., 2011.

    [^1]: http://ai.stanford.edu/~amaas/data/sentiment/

    # License
    CC0 1.0 Universal (CC0 1.0) Public Domain Dedication
    Details in https://creativecommons.org/publicdomain/zero/1.0/

[Korpora] Corpus `nsmc` is already installed at C:\Users\KDP-35\Korpora\nsmc\ratings_train.txt
[Korpora] Corpus `nsmc` is already installed at C:\Users\KD

In [18]:
# pip install gensim

In [19]:
# 2) p297 Word2Vec 모델 학습 

from gensim.models import Word2Vec
## Gensim은 텍스트 관련 문제를 해결하기 위한 강력한 도구로, 주로 주제 모델링과 문서 유사성 분석에 사용 
## 대규모 데이터셋을 처리하는 능력과 다양한 자연어 처리 기능으로 인해, 데이터 과학자, 연구원, 개발자 사이에서 널리 사용

word2vec = Word2Vec(
    sentences = tokens,
    vector_size = 128,
    window = 5,
    min_count = 1,
    sg = 1,
    epochs= 3,
    max_final_vocab = 10000
)

word2vec.save('../10_08/models/word2vec.model')


In [20]:
# 사전 학습된 모델로 임베딩 계층 초기화 

from gensim.models import Word2Vec
import numpy as np

word2vec = Word2Vec.load("../10_08/models/word2vec.model")
init_embeddings = np.zeros((n_vocab, embedding_dim))

for index, token in id_to_token.items():
    if token not in ["<pad>", "<unk>"]:
        init_embeddings[index] = word2vec.wv[token]

## 임베딩 계층은 from_pretrained 메서드로 초기화 ㅇ 
embedding_layer = nn.Embedding.from_pretrained(
    torch.tensor(init_embeddings, dtype=torch.float32)
)

In [22]:
# 기존 SentenceClassifier 클래스 수정 

class SentenceClassifier(nn.Module):
    def __init__(
        self, 
        n_vocab, 
        hidden_dim,
        embedding_dim, 
        n_layers,
        dropout = 0.5,
        bidirectional = True,
        model_type = "lstm",
        pretrained_embedding = None
    ):

        super().__init__()

        self.embedding = nn.Embedding( # 인스턴스 생성 
            num_embeddings=n_vocab,
            embedding_dim=embedding_dim,
            padding_idx=0
        )

        if model_type == "rnn":
            self.model = nn.RNN(
                input_size = embedding_dim,
                hidden_size=hidden_dim,
                num_layers=n_layers,
                bidirectional=bidirectional,
                dropout=dropout,
                batch_first=True,
            )

        elif model_type == "lstm":
            self.model = nn.LSTM(
                input_size = embedding_dim,
                hidden_size = hidden_dim,
                num_layers=n_layers,
                bidirectional=bidirectional,
                dropout=dropout,
                batch_first=True,
            )

        if bidirectional:
            self.classifier = nn.Linear(hidden_dim * 2,1)
        else:
            self.classifier = nn.Linear(hidden_dim, 1)
        self.dropout = nn.Dropout(dropout)

        # 사전 학습된 임베딩(pretrained_embedding)이 None이 아니라면 전달된 값을 임베딩 계층으로 초기화화
        if pretrained_embedding is not None:
            self.embedding = nn.Embedding.from_pretrained(
                torch.tensor(pretrained_embedding, dtype=torch.float32)
            )
        else:
            self.embedding = nn.Embedding(
                num_embeddings=n_vocab,
                embedding_dim=embedding_dim,
                padding_idx = 0
            )


    def forward(self, inputs): # 순방향 메서드
        embeddings = self.embedding(inputs) # 입력받은 정수 인코딩을 임베딩 계층에 통과시켜 임베딩 값을 얻음
        output, _ = self.model(embeddings) # 얻은임베딩 값을 모델에 입력하여 출력값 얻음 
        last_output = output[:,-1,:] 
        last_output = self.dropout(last_output)
        logits = self.classifier(last_output) # 마지막 값만 추출하여 분류기 계층에 전달 
        return logits



In [23]:
# 사전 학습된 임베딩을 사용한 모델 학습 

classifier = SentenceClassifier(
    n_vocab = n_vocab, hidden_dim=hidden_dim, embedding_dim= embedding_dim,
    n_layers= n_layers, pretrained_embedding=init_embeddings
).to(device)

criterion = nn.BCEWithLogitsLoss().to(device)
optimizer = optim.RMSprop(classifier.parameters(), lr=0.001)

epochs = 5
interval = 500

for epoch in range(epochs):
    train(classifier, train_loader, criterion, optimizer, device, interval)
    test(classifier, test_loader, criterion, device)

Train Loss 0 : 0.6916605830192566
Train Loss 500 : 0.6834308970236255
Train Loss 1000 : 0.6704473164412644
Train Loss 1500 : 0.6398692919682217
Train Loss 2000 : 0.6208030407962533
Train Loss 2500 : 0.6073142584802055
Val Loss : 0.5821949984318913, Val Accuracy : 0.7008
Train Loss 0 : 0.6439685821533203
Train Loss 500 : 0.5270143159611258
Train Loss 1000 : 0.5239736239065776
Train Loss 1500 : 0.5144156443921826
Train Loss 2000 : 0.5082371082828976
Train Loss 2500 : 0.5015851934937562
Val Loss : 0.47169405250503615, Val Accuracy : 0.7834
Train Loss 0 : 0.6342604756355286
Train Loss 500 : 0.4666483946546109
Train Loss 1000 : 0.466072138655674
Train Loss 1500 : 0.46236131353667387
Train Loss 2000 : 0.46436137084601103
Train Loss 2500 : 0.4624076225504023
Val Loss : 0.4343377980180442, Val Accuracy : 0.8026
Train Loss 0 : 0.48298218846321106
Train Loss 500 : 0.45271911369469353
Train Loss 1000 : 0.447593104559463
Train Loss 1500 : 0.44408658016808106
Train Loss 2000 : 0.44010486157386675
T

- 사전 학습된 임베딩을 사용하는 것은 모델 성능을 개선할 수 있는 방법 중 하나 
- but 학습 데이터의 양이 많다면 모델의 목적에 맞게 새로운 임베딩 층을 학습하는 것이 더 좋은 결과일 수 ㅇ 
